In [ ]:
# Module imports
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import cm

import sys
import re
import os

from postprocessingutils import eval_stats_reader, running_mean

In [ ]:
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rc('axes', labelsize=28)
plt.rc('font', size=35)
plt.rc('legend', fontsize=22)               # Make the legend/label fonts 
plt.rc('xtick', labelsize=23)               # a little smaller
plt.rc('ytick', labelsize=23)

mrk_size = 12

In [ ]:
# Parameters
nx = 64#16
ny = 65
nz = 64#16
retau = 180
Re_cl = 2100
nu = 1/Re_cl

baseline_dudy_dict = {"180_16x65x16"   : 3.7398798426242075,
                      "180_32x33x32"   : 3.909412638928125,
                      "180_32x65x32"   : 4.325313533321045,
                      "180_64x65x64"   : 3.82829465265046,
                      "180_128x65x128" : 3.82829465265046}
baseline_dudy = baseline_dudy_dict[f"{int(retau)}_" + \
                             f"{nx}x{ny}x{nz}"]

u_tau = np.sqrt(nu*baseline_dudy)
Re_tau = 2*u_tau/nu
tstar = nu/u_tau**2

In [ ]:
# Experiment
ex = {"timestamp" : 1672737751, "nb_env" : 1, "nb_episodes" : 0,
      "dt" : 0.007, "act_freq": 80//4, "nb_interactions" : 3000, "n_agents" : 16, 
      "resolution" : 16, "ctrl_amp" : 0.04285714285714286,
      "ckpt_step" : 6144000, "nb_init" : 1,
      "description" : r"DDPG MinChan, scaled_inputs with u_\tau"}

In [ ]:
## Find the latest episode
from os import walk

main_path = f"../runs/{ex['timestamp']}/env_000"
main_regex = re.compile("main_")
_, _, filenames = next(walk(main_path))
nb_eps = 0
for f in filenames:
    main_check = main_regex.search(f)
    if main_check is not None:
        nb_eps = np.maximum(int(f.split('.')[0].split('_')[1]),nb_eps)
ex["nb_episodes"] = nb_eps

In [ ]:
import os

if os.path.isfile(f"../runs/{ex['timestamp']}/re_hist.npz"):
    with np.load(f"../runs/{ex['timestamp']}/re_hist.npz",allow_pickle=True) as f:
        re_hist = f['re_hist']
    if len(re_hist) == ex["nb_episodes"]-1:
        loading = False
    else:
        loading = True
else:
    loading = True

if loading:
    ## Reading text file
    for i_ep in np.arange(1,ex["nb_episodes"]):
        print(i_ep)
        main_file = f"../../../runs/{ex['timestamp']}/env_000/main_{i_ep}.out"
        re_hist_ = eval_stats_reader(ex,main_file)
        if i_ep == 1:
            re_hist = re_hist_
        else:
            re_hist = np.concatenate((re_hist,re_hist_),axis=0)

    np.savez(f"../runs/{ex['timestamp']}/env_000/re_hist.npz",re_hist=re_hist)
else:
    print("Using saved summary")

# ## Post-process
re_epavg = np.array([np.mean(re_hist[i_ep,1]*2) for i_ep in range(len(re_hist))])
rmean = running_mean(np.concatenate((Re_tau*np.ones((9,)),re_epavg)),N=10)

In [ ]:
%matplotlib widget

## Setup figure
cmap = cm.get_cmap('viridis')
ixys = 512
fig_rep, ax_rep = plt.subplots(2,1,figsize=[6.4,8.4])
ax_rep[0].set(ylabel=r'$Re_{\tau}$',xlabel=r"$t^+$")
ax_rep[1].set(ylabel=r'$Re_{\tau}$',xlabel="Episode")

cm_value = np.linspace(0,1,ex["nb_episodes"])**(1/2)

## Plotting
for i_ep in range(len(re_hist)):
    p_re = ax_rep[0].plot((re_hist[i_ep,0]-np.floor(re_hist[i_ep,0,0]))/tstar,re_hist[i_ep,1]*2,
                          color=cmap(cm_value[-(i_ep+1)]))#, linewidth=2)

# Episode average
p_re = ax_rep[1].plot(re_epavg,linewidth=1,alpha=0.8)
# Running mean
p_re = ax_rep[1].plot(np.linspace(0,len(re_hist),len(rmean)),rmean,'r')

ax_rep[0].set_ylim(top=210)
fig_rep.tight_layout()